# Description

<font size='+2' color='#005F6A'>**Selectable**</font><br>
* Provides a mechanism for selecting items with either **radio**, **multi-select** or **radiox** (unselecteble radio) behavior.
* Uses **Buttons** to represent selectable items.
* Allows for **custom behavior** on first selection and subsequent selections.

```python
class MySelectable (Selectable):
  def setInitState (self):                           self.setItemWidget (Label (value="initial state"))
  def onItemSelect (self, posList, lastSelection):   print ('Your selection:',posList, 'Last selection:',lastSelection)
  def __init__(self, items, behave='radio'):
    super().__init__(item, items, behave=behave, selector=self.onItemSelect)
#
lst = []
display (HBox (children = [MySelectable(lst, behave='multi').widget,
                           MySelectable(lst, behave='multi').widget]) )
```


# Class

In [ ]:
#| default_exp Selectable

In [ ]:
#| hide
#@markdown <font size='+2' color='#005F6A'>**Selectable**</font><br>
#@markdown * Provides a mechanism for selecting items with either **radio**, **multi-select** or **radiox** (unselecteble radio) behavior.
#@markdown * Uses **Buttons** to represent selectable items.
#@markdown * Allows for **custom behavior** on first selection and subsequent selections.
#@markdown <table><tr><td><font size='+2'>
#@markdown
#@markdown ```python
#@markdown class MySelectable (Selectable):
#@markdown   def setInitState (self):                           self.setItemWidget (Label (value="initial state"))
#@markdown   def onItemSelect (self, posList, lastSelection):   print ('Your selection:',posList, 'Last selection:',lastSelection)
#@markdown   def __init__(self, items, behave='radio'):
#@markdown     super().__init__(item, items, behave=behave, selector=self.onItemSelect)
#@markdown #
#@markdown lst = []
#@markdown display (HBox (children = [MySelectable(lst, behave='multi').widget,
#@markdown                            MySelectable(lst, behave='multi').widget]) )
#@markdown ```
#@markdown </td></tr></table>


In [ ]:
#| export
""" Colab Code-Snippets
It uses colab forms to make a top-level setting of the code snippet (optional)
and give a little description. You can run the cell to execute the code snippet.
"""
from ipywidgets import Button, HTML, HBox, VBox, Label, Output
from IPython.display import display

class Selectable:
  """A class representing a selectable item with customizable behavior.
  This code snippet demonstrates the use of a custom selectable widget class
  in a Jupyter notebook. It allows for the selection of items using either
  radio button behavior or multi-select behavior.

  Attributes
  ----------
  * items: list
    A list with Selectable objects.
  * behave: str
    The behavior type ('multi' or 'radio' or 'radiox') for selection.
  * onSelect: function | default: None
    A callback function triggered upon selection.
  * widget: HBox
    The widget representing the selectable item.

  Methods
  -------
  * doBehavior() | -> none
    Executes the selection behavior based on the specified type.
  * _onSelectorClick(b: Button) | -> none
    Handles the click event for the selector button.
  * setItemWidget(widget) | -> none
    Sets the widget to display for the item.
  * setInitState() | -> none
    Initializes the state of the item (to be implemented in subclasses).
  * onFirstSelect(posList) | -> none
    Defines behavior on the first selection (to be implemented in subclasses).

  Behavior
  -------
  * You can set the behavior to 'radio' or 'multi' (radio button or multi selection).
  * On creation it calls the setInitState abstract function where you can build your
    own widget (e.g. HTML-object) and register it with setItemWidget.
  * When you select an item the abstract function onFirstSelect is called so you can
    set your widget in a 'working' state.
  * Every selection triggers the callback function onSelect if set.

  Use cases
  ---------
  * Make your own class, inherit from Selectable and implement the onFirstSelect
    and setInitState function.
  * To react on a select you can give a listener in the super constructor call.
  * To combine several selectable objects to a set you have to give the same list
    on construction.
  * Display an item with the .widget attribute.

  Example
  -------
  class MySelectable (Selectable):
    def setInitState (self):            self.setItemWidget (Label (value="initial state"))
    def onFirstSelect (self, posList):  self.itemWidget.value = "initialized "+self.item
    def onSelection (self, posList):    print ('Your selection:',posList)
    def __init__(self, items, behave='radio'):
      super().__init__(items, behave, self.onSelection)

  # example: two sets of Selectable objects (two lines radio button / one line multi select)
  a, b = [] , []
  #
  display (HBox (children = [MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget] ))
  display (HBox (children = [MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget] ))
  display (HTML(value='<hr>'))
  #
  display (HBox (children = [MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                             MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                             MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget] ))

  """
  def __init__(self, items, behave, selector=None):
    """Initializes the class with given parameters.
    set attributes, generate widgets, bind events and set initial state.
    """
    self.isSelected, self.items, self.behave = False, items, behave
    self.bu_selector = Button(style={'button_color': '#99bfc3'}, layout={'width': '22px', 'height': '22px'})
    self.widget = HBox(children=[self.bu_selector], layout={'min_height': '24px', 'overflow': 'hidden'})
    self.items.append(self)
    self.bu_selector.on_click(self.select)
    self.setInitState()
    self.selector = selector


  def doBehavior (self):
    """Executes the selection behavior based on the specified type.
    """
    self.posList, lastSelect = [], self.lastSelect()

    if self.behave == 'radiox':
      state         = self.items[lastSelect].isSelected
      for item in self.items: item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
      if state:     self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#99bfc3', False
      else:         self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#005F6A', True
      self.posList  = [p for p in range (len(self.items)) if self.items[p].isSelected]

    elif self.behave == 'radio':
      for item in self.items: item.bu_selector.style.button_color, item.isSelected = '#99bfc3', False
      self.bu_selector.style.button_color, self.isSelected, self.posList = '#005F6A', True, [lastSelect]

    elif self.behave == 'multi':
      state         = self.items[lastSelect].isSelected
      if state:     self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#99bfc3', False
      else:         self.items[lastSelect].bu_selector.style.button_color, self.items[lastSelect].isSelected = '#005F6A', True
      self.posList  = [p for p in range (len(self.items)) if self.items[p].isSelected]

    else: raise Exception('unknown behavior: '+self.behave)

  def setItemWidget (self, widget):
    """Sets the widget to display for the item.
    """
    self.itemWidget, self.widget.children = widget, (*self.widget.children, widget)

  def setInitState (self): raise NotImplementedError
  def onSelection (self, posList): raise NotImplementedError

  def select (self, b=None):
    """Handles the click event for the selector button.
    """
    lastSelect=-1
    # search selected item
    for buttonPos in range (len(self.items)):
      if self.items[buttonPos].bu_selector == self.bu_selector: break
    # when found
    if buttonPos < len(self.items):
      # set isLastSelect in all selectables
      for buttonPos in range (len(self.items)): self.items[buttonPos].isLastSelect = False
      self.isLastSelect = True
    # do selectable specific behavior
    self.doBehavior ()
    # call selector
    if self.selector:
      self.selector (self.posList, self.lastSelect())

  def lastSelect (self):
    for buttonPos in range (len(self.items)):
      if self.items[buttonPos].bu_selector == self.bu_selector: break
    if buttonPos < len(self.items): return buttonPos
    else: return -1



In [ ]:
#| hide
# ___________________________________________________________
#|______________________hello_component______________________|
output = Output ()
class MySelectable (Selectable):
  def setInitState (self): self.setItemWidget (Label (value="initial state"))
  def onItemSelect (self, posList, lastSelect=None): 
    output.clear_output()
    with output: print ('Your selection:',posList,'- lastSelection',lastSelect, end='')
  def __init__(self, items, behave='radio'):
    super().__init__(items, behave, self.onItemSelect)

# example: two sets of Selectable objects (two lines radio button / one line multi select)
a, b, c = [] , [], []
#
display (HTML(value='<font size=+1>behave: radio'))
display (HBox (children = [MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget,
                           MySelectable(a).widget, MySelectable(a).widget, MySelectable(a).widget] ))
#
display (HTML(value='<hr><font size=+1>behave: multi'))
display (HBox (children = [MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                           MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget,
                           MySelectable(b, 'multi').widget, MySelectable(b, 'multi').widget] ))

#
display (HTML(value='<hr><font size=+1>behave: radiox'))
display (HBox (children = [MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget,
                           MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget,
                           MySelectable(c, 'radiox').widget, MySelectable(c, 'radiox').widget] ))
display (output)

HTML(value='<font size=+1>behave: radio')

HTML(value='<hr><font size=+1>behave: multi')

HTML(value='<hr><font size=+1>behave: radiox')

Output()

# Test

In [ ]:
#| hide
#@markdown <font size='-1' color='#005F6A'>**UnitTest Selectable**</font><br>

import unittest
from IPython.display import clear_output
only_register_test_Selectable = True # @param {type:"boolean"}
register_and_run_test_Selectable = True # @param {type:"boolean"}

if only_register_test_Selectable or register_and_run_test_Selectable:
  class Test_Selectable (unittest.TestCase):

    def selectedPositions (items): return [pos for pos, i in enumerate(items) if i.isSelected]

    def setUp (self):
      # class for test inherits from Selectable
      class MySelectable (Selectable):
        def setInitState (self):            self.setItemWidget (Label (value="initial state"))
        def onItemSelect (self, posList):   pass
        def __init__(self, item, items, behave='radio'): super().__init__(item, items, behave, self.onItemSelect)


    def test_Selectable_behave_radio (self):
      # example for radio
      lst = []
      MySelectable(lst, 'radio'), MySelectable(lst, 'radio'), MySelectable(lst, 'radio')

      # generation
      self.assertEqual (len(lst), 3)
      self.assertEqual ([i.itemWidget.value for i in lst], ['initial state', 'initial state', 'initial state'])

      # simulate button clicks and test effects
      lst[0].select(lst[0].widget) # click once
      self.assertEqual ([0], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [True,False,False])
      self.assertEqual (lst[0].lastSelect(),0)
      lst[0].select(lst[0].widget) # click two times
      self.assertEqual ([0], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [True,False,False])
      self.assertEqual (lst[0].lastSelect(),0)
      lst[1].select(lst[1].widget) # click once
      self.assertEqual ([1], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,True,False])
      self.assertEqual (lst[1].lastSelect(),1)
      lst[1].select(lst[1].widget) # click two times
      self.assertEqual ([1], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,True,False])
      self.assertEqual (lst[1].lastSelect(),1)
      lst[2].select(lst[2].widget) # click once
      self.assertEqual ([2], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,True])
      self.assertEqual (lst[2].lastSelect(),2)
      lst[2].select(lst[2].widget) # click two times
      self.assertEqual ([2], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,True])
      self.assertEqual (lst[2].lastSelect(),2)

    def test_Selectable_behave_multi (self):
      # example for  multi
      lst = []
      MySelectable(lst, 'multi'), MySelectable(lst, 'multi'), MySelectable(lst, 'multi')

      # simulate button clicks and test effects
      lst[0].select(lst[0].widget)
      self.assertEqual (lst[0].lastSelect(),0)
      self.assertEqual ([0],Test_Selectable.selectedPositions(lst))
      self.assertEqual ([lst[0].isSelected,lst[1].isSelected,lst[2].isSelected], [True,False,False])
      lst[1].select(lst[1].widget)
      self.assertEqual (lst[1].lastSelect(),1)
      self.assertEqual ([0,1],Test_Selectable.selectedPositions(lst))
      self.assertEqual ([lst[0].isSelected,lst[1].isSelected,lst[2].isSelected], [True,True,False])
      lst[2].select(lst[2].widget)
      self.assertEqual (lst[2].lastSelect(),2)
      self.assertEqual ([0,1,2],Test_Selectable.selectedPositions(lst))
      self.assertEqual ([lst[0].isSelected,lst[1].isSelected,lst[2].isSelected], [True,True,True])
      lst[1].select(lst[1].widget)
      self.assertEqual (lst[1].lastSelect(),1)
      self.assertEqual ([0,2],Test_Selectable.selectedPositions(lst))
      self.assertEqual ([lst[0].isSelected,lst[1].isSelected,lst[2].isSelected], [True,False,True])
      lst[2].select(lst[2].widget)
      self.assertEqual (lst[2].lastSelect(),2)
      self.assertEqual ([0],Test_Selectable.selectedPositions(lst))
      self.assertEqual ([lst[0].isSelected,lst[1].isSelected,lst[2].isSelected], [True,False,False])

    def test_Selectable_behave_radiox (self):
      # example for radiox
      lst = []
      MySelectable(lst, 'radiox'), MySelectable(lst, 'radiox'), MySelectable(lst, 'radiox')

      # simulate button clicks and test effects
      lst[0].select(lst[0].widget) # click once
      self.assertEqual (lst[0].lastSelect(),0)
      self.assertEqual ([0], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [True,False,False])
      lst[0].select(lst[0].widget) # click two times
      self.assertEqual (lst[0].lastSelect(),0)
      self.assertEqual ([], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,False])
      lst[1].select(lst[1].widget) # click once
      self.assertEqual (lst[1].lastSelect(),1)
      self.assertEqual ([1], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,True,False])
      lst[1].select(lst[1].widget) # click two times
      self.assertEqual (lst[1].lastSelect(),1)
      self.assertEqual ([], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,False])
      lst[2].select(lst[2].widget) # click once
      self.assertEqual (lst[2].lastSelect(),2)
      self.assertEqual ([2], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,True])
      lst[2].select(lst[2].widget) # click two times
      self.assertEqual (lst[2].lastSelect(),2)
      self.assertEqual ([], Test_Selectable.selectedPositions (lst))
      self.assertEqual ([lst[0].isSelected, lst[1].isSelected, lst[2].isSelected], [False,False,False])


if register_and_run_test_Selectable:
  result = unittest.main(argv=[""], verbosity=2, exit=False).result


test_Selectable_behave_multi (__main__.Test_Selectable.test_Selectable_behave_multi) ... /home/frank/miniforge3/envs/ds11/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
ok
test_Selectable_behave_radio (__main__.Test_Selectable.test_Selectable_behave_radio) ... ok
test_Selectable_behave_radiox (__main__.Test_Selectable.test_Selectable_behave_radiox) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.097s

OK


# Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()